In [1]:
import pandas as pd
import geopandas as gpd
from siuba import *
import numpy as np

In [2]:
import zipfile

In [3]:
from calitp_data_analysis import get_fs

In [4]:
fs = get_fs()

In [5]:
# ! pip install pygris

In [6]:
import _utils
# import importlib
# importlib.reload(_utils)

In [7]:
GCS_PATH = 'gs://calitp-analytics-data/data-analyses/sb125/vmt_transit_sketch/'

# Read and group Replica data

In [8]:
def read_group_replica(zip_path):
    '''
    zip_path: path to zip file containing a Replica trips export csv    
    '''
    def parse_csv(zipfile):
        csvs = [f for f in z.namelist() if f[-3:] == 'csv']
        assert len(csvs) == 1
        with z.open(csvs[0]) as f:
            df = pd.read_csv(f)
        return df
    
    if zip_path[:3] == 'gs:':
        with fs.open(zip_path) as f:
            with zipfile.ZipFile(f) as z:
                df = parse_csv(z)
    else:
        with zipfile.ZipFile(f) as z:
            df = parse_csv(z)
            

    df = (df >> filter(_.primary_mode.isin(['private_auto', 'auto_passenger', 'on_demand_auto', 'public_transit']))
     >> select(-_.origin_trct_2020, -_.activity_id)
         )
    df['is_auto'] = df.primary_mode.str.contains('auto')
    return df
    grouped = (df >> group_by(_.origin_trct_fips_2020, _.is_auto)
                  >> summarize(n = _.shape[0], p50_distance = _.trip_distance_miles.quantile(.5),
                               p75_distance = _.trip_distance_miles.quantile(.75),
                               p90_distance = _.trip_distance_miles.quantile(.9),
                               total_miles = _.trip_distance_miles.sum(),                               
                              )
        )

    return grouped

In [9]:
f'{GCS_PATH}replica_raw/corridors/replica-fresno-trips_dataset.zip'

'gs://calitp-analytics-data/data-analyses/sb125/vmt_transit_sketch/replica_raw/corridors/replica-fresno-trips_dataset.zip'

In [10]:
fresno_raw = read_group_replica(f'{GCS_PATH}replica_raw/corridors/replica-fresno-trips_dataset.zip')

In [11]:
fresno_raw.describe()

,trip_duration_minutes,trip_distance_miles,origin_trct_fips_2020
count,156196.000000,156196.000000,1.561960e+05
mean,8.027331,2.489494,6.019003e+09
std,6.041116,2.305581,1.731910e+03
min,0.000000,0.100000,6.019000e+09
25%,3.000000,0.800000,6.019002e+09
50%,7.000000,1.600000,6.019004e+09
75%,11.000000,3.500000,6.019005e+09
max,93.000000,17.800000,6.019005e+09


In [12]:
# (wilshire_raw >> filter(_.is_auto)).trip_distance_miles.hist()

In [13]:
# (wilshire_raw >> filter(_.is_auto, _.trip_distance_miles < 4)).trip_distance_miles.hist()

## grouping

In [14]:
all_regions = ['central_a', 'central_b', 'north', 'la_north',
              'la_south', 'sandiego', 'socal_a', 'socal_b']

In [15]:
# grouped = pd.DataFrame()
# for region in ['eureka']:
#     print(region)
#     #  note replica filename includes date of download...
#     df = read_group_replica(f'replica_raw/corridors/replica-{region}-trips_dataset.zip')
#     grouped = pd.concat([grouped, df])

In [16]:
# grouped.to_parquet('intermediate/eureka_grouped.parquet')

In [17]:
# grouped = pd.DataFrame()
# for region in all_regions:
#     print(region)
#     df = read_group_replica(f'replica_raw/replica-{region}-trips_dataset.zip')
#     grouped = pd.concat([grouped, df])

# grouped.to_parquet('intermediate/replica_grouped.parquet')

# Read back in grouped data

* number of trips, median distance, and total miles travelled by auto yes/no and Census tract
* TODO non-manual regions :)

In [18]:
# grouped = pd.DataFrame()
# for region in all_regions:
#     grouped = pd.concat([grouped, pd.read_parquet(f'{GCS_PATH}intermediate/replica-{region}-trips_dataset.parquet')])

In [19]:
# grouped

In [20]:
from calitp_data_analysis import utils

In [21]:
# utils.geoparquet_gcs_export?

In [22]:
# utils.geoparquet_gcs_export(grouped, f'{GCS_PATH}intermediate/', 'replica_grouped')

In [23]:
replica_grouped = pd.read_parquet(f'{GCS_PATH}intermediate/replica_grouped.parquet')

In [24]:
tracts_feeds = gpd.read_parquet(f'{GCS_PATH}intermediate/feeds_tract_geo.parquet')

In [25]:
tracts_feeds.GEOID = tracts_feeds.GEOID.astype('int64')

In [26]:
tracts_feeds = tracts_feeds >> distinct(_.GEOID, _.geometry)

In [27]:
# replica_grouped = pd.read_parquet(f'intermediate/wilshire_grouped.parquet')

In [28]:
18062 / 1.6

11288.75

In [29]:
# (replica_grouped >> filter(_.is_auto))

In [30]:
def process_grouped_data(replica_df, tracts_feeds_df):
    '''
    replica_df: df from read_group_replica
    tracts_feeds_df: gdf from stops_by_tract_agency
    '''
    auto_trip_counts = (replica_df >> filter(_.is_auto)
                        >> group_by(_.origin_trct_fips_2020)
                        >> summarize(auto_trips = _.n.sum())
                       )
    replica_df.is_auto = replica_df.is_auto.map(lambda x: 'yes' if x else 'no')
    
    df2 = replica_df >> spread('is_auto', 'p50_distance') >> select(-_.n, -_.total_miles)
    df2 = df2.rename(columns={'no': 'p50_mi_transit', 'yes': 'p50_mi_auto'})
    df3 = replica_df >> spread('is_auto', 'total_miles') >> select(-_.n, -_.p50_distance)
    df3 = df3.rename(columns={'no': 'total_mi_transit', 'yes': 'total_mi_auto'})
    df2 = df2 >> inner_join(_, df3, on = 'origin_trct_fips_2020')
    df2 = df2 >> inner_join(_, auto_trip_counts, on = 'origin_trct_fips_2020')
    
    if 'p50_mi_transit' not in df2.columns:
        df2['p50_mi_transit'] = np.nan
        df2['total_mi_transit'] = np.nan
    df2 = (df2 >> group_by(_.origin_trct_fips_2020)
           >> summarize(p50_mi_transit = _.p50_mi_transit.max(), p50_mi_auto = _.p50_mi_auto.max(),
                        total_mi_transit = _.total_mi_transit.max(),
                        total_mi_auto = _.total_mi_auto.max(),
                        total_trips_auto = _.auto_trips.sum()
                       )
          )
    
    df2['p50_transit_longer'] = df2['p50_mi_transit'] > df2['p50_mi_auto']
    # df2['total_transit_higher'] = df2['total_mi_transit'] > df2['total_mi_auto'] #  useless
    tracts_with_stops = tracts_feeds_df.GEOID
    df2['no_transit_replica'] = df2.total_mi_transit.isna()
    df2['no_transit'] = ~df2.origin_trct_fips_2020.isin(tracts_with_stops)
    df2['total_mi'] = df2.total_mi_transit + df2.total_mi_auto
    no_transit = df2 >> group_by(_.no_transit) >> summarize(total_mi_auto = _.total_mi_auto.sum())
    total_reduction = df2.total_mi_auto.sum() * .25 #  CARB 25% VMT Decrease
    
    # rescaling after no transit tracts
    actual_reduction = total_reduction / no_transit.iloc[0, 1]
    print(actual_reduction)
    df2.p50_mi_transit = df2.p50_mi_transit.fillna(df2.p50_mi_auto)
    df2['new_transit_mi'] = df2.total_mi_auto * actual_reduction
    df2['projected_new_transit_trips'] = df2.new_transit_mi // df2.p50_mi_transit
    
    return df2

In [31]:
processed_df = process_grouped_data(replica_grouped, tracts_feeds)

0.2778337212327877


In [32]:
processed_df >> head(3)

,origin_trct_fips_2020,p50_mi_transit,p50_mi_auto,total_mi_transit,total_mi_auto,total_trips_auto,p50_transit_longer,no_transit_replica,no_transit,total_mi,new_transit_mi,projected_new_transit_trips
0,6001400100,4.2,6.9,5185.3,165410.8,53636,False,False,False,170596.1,45956.698096,10942.0
1,6001400200,4.1,3.4,3638.1,58763.4,28600,True,False,False,62401.5,16326.454094,3982.0
2,6001400300,4.7,3.7,6130.7,122736.5,57740,True,False,False,128867.2,34100.338526,7255.0


In [33]:
no_transit = processed_df >> group_by(_.no_transit) >> summarize(total_mi_auto = _.total_mi_auto.sum())
no_transit

,no_transit,total_mi_auto
0,False,1.162508e+09
1,True,1.294277e+08


In [34]:
# no_transit_pct = no_transit.iloc[1, 1] / no_transit.iloc[0, 1]
# f'{round(no_transit_pct*100, 0)} percent of VMT in tracts with no transit per GTFS Warehouse stops'

In [35]:
processed_df.p50_transit_longer.value_counts()

True     6571
False    2534
Name: p50_transit_longer, dtype: int64

In [36]:
tract_geo = _utils.get_tract_geoms()

Using FIPS code '06' for input 'CA'


In [37]:
ca_uzas = gpd.read_parquet(f'{GCS_PATH}intermediate/ca_uza.parquet')

In [59]:
# census_pop = gpd.read_file(f'./census_ntd/DECENNIALPL2020.P1_2024-02-01T163251.zip')

# census_cleaned = census_pop.iloc[2:,:][['GEO_ID', 'P1_001N']]

# census_cleaned.to_parquet(f'{GCS_PATH}census_ntd/DECENNIALPL2020.parquet')

In [60]:
census_cleaned = pd.read_parquet(f'{GCS_PATH}census_ntd/DECENNIALPL2020.parquet')

In [61]:
def attach_tracts_pop(processed_df, tract_geo, ca_uzas, census_cleaned):
    
    tract_geo.GEOID = tract_geo.GEOID.astype('int64')
    gdf = (tract_geo >> inner_join(_, processed_df, on = {'GEOID': 'origin_trct_fips_2020'})
                     >> select(-_.origin_trct_fips_2020))
    uza_joined = gpd.sjoin(gdf, ca_uzas, how = 'left')
    
    tract_pop = census_cleaned
    tract_pop.GEO_ID = tract_pop.GEO_ID.map(lambda x: x.split('US')[1])
    tract_pop.GEO_ID = tract_pop.GEO_ID.astype('int64')
    tract_pop = tract_pop >> select(_.total_pop == _.P1_001N, _.GEOID == _.GEO_ID)
    uza_joined = uza_joined >> inner_join(_, tract_pop, on = 'GEOID')
    uza_joined.total_pop = uza_joined.total_pop.astype('int64')
    uza_joined['new_trips_per_capita'] = uza_joined.projected_new_transit_trips / uza_joined.total_pop
    
    return uza_joined

In [63]:
uza_joined = attach_tracts_pop(processed_df, tract_geo, ca_uzas, census_cleaned)

In [ ]:
# for region in ['sanpablo', 'eureka', 'fresno', 'wilshire']:
#     replica_grouped = pd.read_parquet(f'intermediate/{region}_grouped.parquet')
#     processed_df = process_grouped_data(replica_grouped, tracts_feeds)
#     uza_joined = attach_tracts_pop(processed_df)
#     display(uza_joined >> head(3))
#     uza_joined.to_parquet(f'outputs/{region}_trips_with_uza.parquet')

In [ ]:
# uza_joined = uza_joined >> filter(_.P1_001N != 0) # remove tracts where nobody lives

In [68]:
uza_joined = (uza_joined
              >> select(-_.index_right)
              )

In [71]:
uza_joined['vmt_quantile'] = pd.qcut(uza_joined.total_mi_auto, 4, labels = ['p25', 'p50', 'p75', 'p100'])

In [72]:
!mkdir export

In [75]:
uza_joined.vmt_quantile = uza_joined.vmt_quantile.astype(str)

In [79]:
utils.geoparquet_gcs_export(uza_joined, f'{GCS_PATH}outputs/', 'new_trips_with_uza')

In [77]:
uza_joined.to_file('./export/vmt_with_quantiles.shp')

/tmp/ipykernel_532/1657757500.py:1: UserWarning: Column names longer than 10 characters will be truncated when saved to ESRI Shapefile.
  uza_joined.to_file('./export/vmt_with_quantiles.shp')
